# In this notebook, we are going to create a model for detecting anomalies using the autoencoder method.

make the dataset and train the model on it

In [1]:
# import the lib
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Generate random normal data
normal_data = np.random.normal(0, 1, size=(1000, 32))
normal_data.shape

(1000, 32)

In [3]:
# set the train data and the train labels
train_data = normal_data
train_labels = np.array(len(normal_data)*[0])  # make 0 array as length same as the nrmal data length

In [4]:
train_labels[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
# split the data set to the train and the test set
x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [6]:
# test the x_train shape
x_train.shape

(800, 32)

In [7]:
# set the imput shape
input_dim = x_train.shape[1]

# set the encoder
encoder = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu')
])

# set the decoder
decoder = models.Sequential([
    layers.Input(shape=(16,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

# set the autoencoder usisng the encoder and the decoder
autoencoder = models.Sequential([
    encoder,
    decoder
])

# compile the auto encoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [8]:
# set the early stop callback
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = "val_loss", patience = 10, restore_best_weights = True)


# fit the auto encoder model
autoencoder.fit(x_train,
                x_train,
                epochs= 200,
                batch_size=128,
                validation_data=(x_test, x_test),
                callbacks = early_stop)


Epoch 1/200
7/7 [==============================] - 2s 45ms/step - loss: 1.2482 - val_loss: 1.2484
Epoch 2/200
7/7 [==============================] - 0s 12ms/step - loss: 1.2322 - val_loss: 1.2273
Epoch 3/200
7/7 [==============================] - 0s 12ms/step - loss: 1.2038 - val_loss: 1.1884
Epoch 4/200
7/7 [==============================] - 0s 12ms/step - loss: 1.1554 - val_loss: 1.1292
Epoch 5/200
7/7 [==============================] - 0s 12ms/step - loss: 1.0931 - val_loss: 1.0704
Epoch 6/200
7/7 [==============================] - 0s 16ms/step - loss: 1.0459 - val_loss: 1.0394
Epoch 7/200
7/7 [==============================] - 0s 18ms/step - loss: 1.0250 - val_loss: 1.0258
Epoch 8/200
7/7 [==============================] - 0s 14ms/step - loss: 1.0125 - val_loss: 1.0103
Epoch 9/200
7/7 [==============================] - 0s 14ms/step - loss: 1.0007 - val_loss: 1.0045
Epoch 10/200
7/7 [==============================] - 0s 17ms/step - loss: 0.9976 - val_loss: 1.0027
Epoch 11/200
7/7 [=

In [9]:
# evaluate the autoencoder mdoel
autoencoder.evaluate(x_test[y_test == 0], x_test[y_test == 0])

7/7 [==============================] - 0s 2ms/step - loss: 0.8836


0.8836498260498047

Make another dataset and detect the anomalys

In [82]:
# now lets make a anomaly and the normaly (wat we are trainded for) data set for seprate the anomaly data

normal_data_2  = np.random.normal(0, 1, size = (1000, 32))
anomaly_data_2 = np.random.normal(7, 2, size = (300, 32))

In [83]:
# mix the anomal and the normal data

data = np.vstack([normal_data_2, anomaly_data_2])
data.shape

(1300, 32)

In [84]:
# set the labels
# lets difine as normal data as 0 and anomaly data 1
labels = np.array([0]*len(normal_data_2) + [1]*len(anomaly_data_2))

In [85]:

# Calculate reconstruction errors for all data
reconstructed_data = autoencoder.predict(data)
mse = np.mean(np.power(data - reconstructed_data, 2), axis=1)

print(len(mse))

41/41 [==============================] - 0s 2ms/step
1300


In [86]:
# Set a threshold for anomaly detection (you can adjust this threshold)
threshold = 1.7  # this is we can adjust how we want

# Classify anomalies based on the threshold
predictions = (mse > threshold).astype(int)
predictions_2 = [x for x in predictions if x == 1]

print(len(predictions))
print(len(predictions_2))

1300
301


In [87]:
print(predictions[800:])


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

We can see that the model has successfully identified the anomalies correctly. Therefore, our model can predict the anomalies in that dataset accurately using the 1.7 threshold.

end